In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob

import pickle

from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
def generate_prompt_predict_possible_joins(target_description, candidate_description):

    system_msg = f"""
            Given one target column description and one candidate column description, predict if they could be joined, Answer with only 'Yes' or 'No'.

            Task: Look carefully at the target column description and the candidate column description and use this information to identify patterns and 
            relationships between the descriptions, the result must be Yes or No depending if the pair is JOINable.
            
            Additional info: A JOIN in relational databases is an operation that retrieves related rows from two tables by linking them based on related 
            columns between them.
            
            Instructions: 
                1. Look at the target description given to you. 
                2. Look at the candidate description in detail. 
                3. Predict if the target column description belongs to a column that may be used in join. 
                4. Predict if the target column description and the candidate column description are JOINable. Disregard the column names.
                
            Example 1:
                Target description: this column represents a worker's id.
                Candidate description: 'the column worker_id represents the worker's id'.
                Answer: Yes
            """
    
    user_msg = f"""Target description:     {target_description}
                   Candidate description:  {candidate_description}
                   Answer: """.strip()
    
    return system_msg, user_msg

In [5]:
cta_matched_joinable_tables = pd.read_csv('Description_test/table_descriptions_test_2025_01_05/junio_JD_matches_joinable_tables.csv')
cta_matched_joinable_tables = cta_matched_joinable_tables[cta_matched_joinable_tables['JOINABLE_TD_v2'] == 'Yes']
cta_matched_joinable_tables.head()

,LEFT_TABLE,LEFT_COLUMN,LEFT_DESCRIPTION,RIGHT_TABLE,RIGHT_COLUMN,RIGHT_DESCRIPTION,JOINABLE,JOINABLE_TD,JOINABLE_TD_v2
0,community-centres,ADDRESS,The full postal address of the community garde...,community-gardens-and-food-trees,MERGED_ADDRESS,The ADDRESS column provides the street address...,Yes,Yes,Yes
1,community-centres,Geo Local Area,The neighbourhood or broader geographical area...,community-gardens-and-food-trees,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes
2,community-centres,ADDRESS,This column contains the street address of the...,cultural-spaces,ADDRESS,The ADDRESS column provides the street address...,Yes,Yes,Yes
3,community-centres,Geo Local Area,This column identifies the local area or neigh...,cultural-spaces,LOCAL_AREA,The Geo Local Area column identifies the gener...,Yes,Yes,Yes
6,community-centres,Geo Local Area,"In this column, the local areas or neighborhoo...",libraries,Geo Local Area,The Geo Local Area column identifies the gener...,Yes,Yes,Yes


In [7]:
prompts = []

for i in tqdm(range(len(cta_matched_joinable_tables))):
    
    target_table_description = cta_matched_joinable_tables.iloc[i, 2]
    target_column_descriptions = cta_matched_joinable_tables.iloc[i, 5]
    
    prompt = generate_prompt_predict_possible_joins(target_table_description, target_column_descriptions)
    prompts.append(prompt)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:00<00:00, 9595.18it/s]


In [8]:
prompts[0]

("\n            Given one target column description and one candidate column description, predict if they could be joined, Answer with only 'Yes' or 'No'.\n\n            Task: Look carefully at the target column description and the candidate column description and use this information to identify patterns and \n            relationships between the descriptions, the result must be Yes or No depending if the pair is JOINable.\n            \n            Additional info: A JOIN in relational databases is an operation that retrieves related rows from two tables by linking them based on related \n            columns between them.\n            \n            Instructions: \n                1. Look at the target description given to you. \n                2. Look at the candidate description in detail. \n                3. Predict if the target column description belongs to a column that may be used in join. \n                4. Predict if the target column description and the candidate column

In [9]:
np.save('Description_test/table_descriptions_test_2025_01_05/Column_JOIN_Prompts/column_JD.npy', prompts)